In [13]:
import os, sys
from PIL import Image
import random
import numpy as np
from pathlib import Path

In [89]:
#Change path here:
path = "/Users/robertwalter/Desktop/RCI.2/2_MLMI/6_datasets/MontgomerySet_Versions/Version8"
#Change Names of output folder and rebinarized testImage labels here:
folder_out = "/out_image_bin"
folder_testlabels = "/testMasks_bin"

def calc_singleConfusion(out_array,label_array):
    if out_array.size == label_array.size:
        
        #Confusion Matrix with [tp,fp,tn,fn]
        confMatrix = [0] *4
        
        #print(len(out_array))
        #print(len(out_array[0]))
        for i in range(len(out_array)):
            for j in range(len(out_array[0])):
                
                if (label_array[i][j] != 0 and label_array[i][j] != 1) or (out_array[i][j] != 0 and out_array[i][j] != 1):
                    print("Image not binary")
                    return confMatrix
                else:   
                    #Negatives
                    if label_array[i][j] == 0:
                        #True-Negatives
                        if out_array[i][j] == label_array[i][j]:
                            confMatrix[2] += 1
                        #False-Negative
                        else:
                            confMatrix[3] += 1
                    #Positives
                    elif label_array[i][j] == 1:
                        #True-Positives
                        if out_array[i][j] == label_array[i][j]:
                            confMatrix[0] += 1
                        #False-Positives
                        else:
                            confMatrix[1] += 1

    else:
        print("Image Dimensions dont fit!")
        
    return confMatrix
        
def prepImage(img_name):
    img_out = Image.open(path+folder_out+"/"+img_name)
    img_label = Image.open(path+folder_testlabels+"/"+img_name)
    
    #check patient condition
    if img_name.endswith("0.png"):
        condFlag = 0
    elif img_name.endswith("1.png"):
        condFlag = 1   
    else:
        print(img_name+" patient condition not clear")
    
    arr_out = np.array(img_out)
    arr_label = np.array(img_label)
    #print(arr_out)
    return arr_out, arr_label, condFlag



def main():
    #Counter [all,normal,abnormal]
    counter = [0] *3
    #Confusion Matrix 2x2x[all,normal,abnormal]
    confMatrix_complete = np.zeros((4,3))

    for img in os.listdir(path+folder_out):

        if img.endswith(".png"): 

            my_file = Path(path+folder_testlabels+"/"+img)
            if my_file.is_file():
                counter[0] += 1
                arr_out, arr_label, condFlag = prepImage(img)
                confMatrix = calc_singleConfusion(arr_out, arr_label) 
                #print(confMatrix)
                #all
                confMatrix_complete[:,0] += confMatrix
                
                #normal
                if condFlag==0:
                    counter[1] +=1
                    confMatrix_complete[:,1] += confMatrix
                #abnormal
                else:
                    counter[2] +=1
                    confMatrix_complete[:,2] += confMatrix
                    
                #Averaging                
 
          
            else:
                print(img+" Mask does not exist")
                
    confMatrix_complete[:,0] = confMatrix_complete[:,0]/counter[0]             
    confMatrix_complete[:,1] = confMatrix_complete[:,1]/counter[1]  
    confMatrix_complete[:,2] = confMatrix_complete[:,2]/counter[2]
    np.set_printoptions(precision=3)
    print("Confusion Matrix [tp,fp,tn,fn] average over all images ({}): {}".format(counter[0],confMatrix_complete[:,0]))
    print("Confusion Matrix [tp,fp,tn,fn] average over normal images ({}): {}".format(counter[1],confMatrix_complete[:,1]))  
    print("Confusion Matrix [tp,fp,tn,fn] average over abnormal images ({}): {}".format(counter[2],confMatrix_complete[:,2]))
                
                

In [86]:
main()

Confusion Matrix [tp,fp,tn,fn] average over all images (11): [ 44803.545    201.636 125879.909   1914.909]
Confusion Matrix [tp,fp,tn,fn] average over normal images (6): [ 40393.333    212.667 130825.333   1368.667]
Confusion Matrix [tp,fp,tn,fn] average over abnormal images (5): [ 50095.8    188.4 119945.4   2570.4]


In [91]:
#testing
arr1 = np.array([[1,0,0],[1,1,1],[1,0,1]])
arr2 = np.array([[1,0,0],[1,0,1],[1,0,1]])
ConfMat = calc_singleConfusion(arr1,arr2)
print(ConfMat)

[5, 0, 3, 1]
